<a href="https://colab.research.google.com/github/constantinpape/dl-teaching-resources/blob/main/exercises/classification/4_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN on CIFAR10

In this exercise we will train our first convolutional neural network (CNN) on the cifar 10 dataset. Unlike the previous models we have used, logistic regression and multi-layer perceptrons, CNNs learn filters and classification jointly.

## Preparation

In [ ]:
# load tensorboard extension
%load_ext tensorboard

In [ ]:
# import torch and other libraries
import os
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from tqdm import trange

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter

In [ ]:
!pip install cifar2png

In [ ]:
# check if we have gpu support
# colab offers free gpus, however they are not activated by default.
# to activate the gpu, go to 'Runtime->Change runtime type'. 
# Then select 'GPU' in 'Hardware accelerator' and click 'Save'
have_gpu = torch.cuda.is_available()
# we need to define the device for torch, yadda yadda
if have_gpu:
    print("GPU is available")
    device = torch.device('cuda')
else:
    print("GPU is not available, training will run on the CPU")
    device = torch.device('cpu')

In [ ]:
# run this in google colab to get the utils.py file
!wget https://raw.githubusercontent.com/constantinpape/training-deep-learning-models-for-vison/master/day1/utils.py 

In [ ]:
# we will reuse the training function, validation function and
# data preparation from the previous notebook
import utils

In [ ]:
cifar_dir = './cifar10'
!cifar2png cifar10 cifar10

In [ ]:
categories = os.listdir('./cifar10/train')
categories.sort()

In [ ]:
# get training and validation data
train_dataset, val_dataset = utils.make_cifar_datasets(cifar_dir)

## CNN

Define the model architecture for our first CNN. The network is made up of the following components:
- convolutional layers that convolve their input with a learnable filter, using less parameters than a fully connected layer while keeping spatial context
- max pooling that halves the image size
- fully connected layers at the end of the network to output a class prediction vector for the input

Note that both convolutional layers and pooling operations change the spatial
size of the data. You can find the formulas for this in the torch class descriptions: [nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#conv2d), [nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html).
See the comments in network definition below for an example.

In [ ]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes

        # the convolutions
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)

        # the fully connected part of the network
        # after applying the convolutions and poolings, the tensor
        # has the shape 24 x 6 x 6, see below
        self.fc = nn.Sequential(
            nn.Linear(24 * 6 * 6, 120),
            nn.ReLU(),
            nn.Linear(120, 60),
            nn.ReLU(),
            nn.Linear(60, self.n_classes)
        )
        self.activation = nn.LogSoftmax(dim=1)

    def apply_convs(self, x):
      # input image has shape 3 x  32 x 32
      x = self.pool(F.relu(self.conv1(x)))
      # shape after conv: 12 x 28 x 28
      # shape after pooling: 12 x 14 X 14
      x = self.pool(F.relu(self.conv2(x)))
      # shape after conv: 24 x 12 x 12
      # shape after pooling: 24 x 6 x 6
      return x
    
    def forward(self, x):
        x = self.apply_convs(x)
        x = x.view(-1, 24 * 6 * 6)
        x = self.fc(x)
        x = self.activation(x)
        return x

In [ ]:
# instantiate the model
model = CNN(n_classes=10)
model.to(device)

In [ ]:
# instantiate loaders, loss, optimizer and tensorboard

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=25)

optimizer = Adam(model.parameters(), lr=1.e-3)

loss_function = nn.NLLLoss()
loss_function.to(device)

tb_logger = SummaryWriter('runs/log_cnn')
%tensorboard --logdir runs

In [ ]:
n_epochs = 10
for epoch in trange(n_epochs):
    utils.train(model, train_loader, loss_function, optimizer,
                device, epoch, tb_logger=tb_logger)
    step = (epoch + 1) * len(train_loader)
    utils.validate(model, val_loader, loss_function,
                   device, step,
                   tb_logger=tb_logger)

In [ ]:
# evaluate the model on test data
test_dataset = utils.make_cifar_test_dataset(cifar_dir)
test_loader = DataLoader(test_dataset, batch_size=25)
predictions, labels = utils.validate(model, test_loader, loss_function,
                                     device, step=0, tb_logger=None)

In [ ]:
import matplotlib.pyplot as plt
print("Test accuracy:")
accuracy = metrics.accuracy_score(labels, predictions)
print(accuracy)

fig, ax = plt.subplots(1, figsize=(8, 8))
utils.make_confusion_matrix(labels, predictions, categories, ax)

## Checkpoints and adaptive learning rate

The model weights can be saved in order to load the model again and run prediction in a different application. If in addiation the optimizer state is saved the model training can also be resumed from the same state.

Another important aspect of training neural networks is adapting the learning rate during training, which can increase model performance by converging to better optima. Here, we will condition the learning rate decrease on the validation accuracy using [torch.ReduceLROnPlateau](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.ReduceLROnPlateau).

In [ ]:
# functions to save and load a checkpoint (= serialized model and optimizer state +
# additional metadata)
def save_checkpoint(model, optimizer, epoch, save_path):
    # the state_dict of the model and the optimizer contain the current
    # state of the model parameters and the optimizer
    # it's only necessary to save the latter to resume training
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, save_path)


def load_checkpoin(save_path, model, optimizer):
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict']) 
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    return model, optimizer, epoch

In [ ]:
# instantiate loaders, loss, optimizer and tensorboard

# instantiate the model
model = CNN(10)
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=25)

optimizer = Adam(model.parameters(), lr=1.e-3)

loss_function = nn.NLLLoss()
loss_function.to(device)

tb_logger = SummaryWriter("runs/log_cnn1")

In [ ]:
n_epochs = 15

# we use the best checkpoint as measured by the validation accuracy
best_accuracy = 0.
best_epoch = 0

# monitor the validation accuracy and decrease the learning rate when
# it starts to plateau
# NOTE: it's usually better to choose a higher patience value than a single epoch,
# we choose this value here in order to observe the changes when only training for 15 epochs
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer,
                              mode="max",  # we evaluate based on accuracy, for which higher values are better
                              factor=0.5,  # half the learning rate
                              patience=1)  # number of epochs without improvement after which we reduce the lr

for epoch in trange(n_epochs):
    utils.train(model, train_loader, loss_function, optimizer,
                device, epoch, tb_logger=tb_logger)
    step = (epoch + 1) * len(train_loader)

    pred, labels = utils.validate(model, val_loader, loss_function, device, step,
                                  tb_logger=tb_logger)
    val_accuracy = metrics.accuracy_score(labels, pred)
    scheduler.step(val_accuracy)
    
    # otherwise, check if this is our best epoch
    if val_accuracy > best_accuracy:
        # if it is, save this check point
        best_accuracy = val_accuracy
        best_epoch = epoch
        # print("Saving best checkpoint for epoch", epoch)
        save_checkpoint(model, optimizer, epoch, "./best_checkpoint.tar")

print("The best checkpoint is number", best_epoch) 

In [ ]:
# load the model for the best checkpoint and evaluate it on the test data.
# how do the results compare to before?
# model, _, _ = load_checkpoint('./best_checkpoint.tar', model, optimizer)

test_dataset = utils.make_cifar_test_dataset(cifar_dir)
test_loader = DataLoader(test_dataset, batch_size=25)
predictions, labels = utils.validate(model, test_loader, loss_function,
                                     device, step=0, tb_logger=None)

In [ ]:
import matplotlib.pyplot as plt
print("Test accuracy:")
accuracy = metrics.accuracy_score(labels, predictions)
print(accuracy)

fig, ax = plt.subplots(1, figsize=(8, 8))
utils.make_confusion_matrix(labels, predictions, categories, ax)

## Visualize learned filters

The output of convolutional filters is still spatial, so it can be intepreted as images. Here, we inspect some of the filter responses of our network in order to visualize what features these filters pick up on.

In [ ]:
# apply the first convolutional filter of our model 
# to the first image from the training dataset
model.to(torch.device('cpu'))
model.eval()
im = train_dataset[0][0][None]

In [ ]:
conv1_response = model.conv1(im).detach().numpy()[0]
print(conv1_response.shape)

In [ ]:
# visualize the filters in the first layer
import matplotlib.pyplot as plt
n_filters = 8
fig, axes = plt.subplots(1, 1 + n_filters, figsize=(16, 4))
im = im[0].numpy().transpose((1, 2, 0))
axes[0].imshow(im)
for chan_id in range(n_filters):
    axes[chan_id + 1].imshow(conv1_response[chan_id], cmap='gray')

## Tasks and Questions

Tasks:
- Train another network with more convolutional layers / poolings and observe how the performance and training behaviour changes.
- Visualize the convolutional filters for some of the deeper layers in your conv net.

Questions:
- How did the different models you have trained in this exercise perform? How do they compare to the non-convolutional models trained before?
- How do you interpret the convolutional filters based on their visualisations? Can you see differences between the filters in the first and deeper layers?

Advanced:
- Construct a CNN that can be applied to input images of arbitrary size. Hint: Have a look at [nn.AdaptiveAveragePool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html).